# Overview of MEG/EEG analysis with MNE-Python

## 1. Loading data

We can use the `mne.io.read_raw_fif()` function to load data. *MNE-Python data structures* are based around the *FIF* file format from Neuromag, but there are also *other reader functions* for a [wide variety of other data formats](https://mne.tools/stable/documentation/implementation.html#data-formats).
<br />
*MNE-Python* can also download and manage variety of [public available datasets via its interfaces](https://mne.tools/stable/documentation/datasets.html#datasets).
<br />
<br />
Let's load one of the example dataset, "[Samples](https://mne.tools/stable/documentation/datasets.html#sample-dataset)", which contains EEG and MEG data from one subject performing an audiovisual experiment, along with structural MRI scans for that subject.
<br />
The `mne.datasets.sample.data_path()` interface function will automatically download the dataset if it isn’t found in one of the expected locations or we can specify where to download the data by passing the `path=` parameter. The interface function then return the directory path to the dataset.
> Note: The "Samples" dataset contains two datasets `sample_audvis_raw.fif` which  is an unfiltered version and `sample_audvis_filt-0-40_raw.fif` which is a filtered and downsampled version of the data.

In [ ]:
import numpy as np
import os

import mne
mne.viz.set_3d_backend("notebook")

In [ ]:
# download the data 
sample_data_path = mne.datasets.sample.data_path(
    path="./"
)

print(f"\n\nsPath to the dataset: {sample_data_path}")

In [ ]:
sample_version = "sample_audvis_raw.fif"
sample_audvis_raw_path = os.path.join("MEG/sample/", sample_version)
sample_data_path = os.path.join(sample_data_path, sample_audvis_raw_path)

print(f"Loading {sample_version} dataset from the path:\n\t{sample_data_path}\n\n")
sample_raw = mne.io.read_raw_fif(sample_data_path)

We can see, by deafult, `read_raw_fif()` displays some info about the dataset it has loaded. Howerver, we can also get some basic details of a [`Raw` dataset object](https://mne.tools/stable/generated/mne.io.Raw.html#mne.io.Raw) by printing it or by calling its `.info` property:

In [ ]:
print(sample_raw)
print("\n~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~\n")
print(sample_raw.info)

The `Raw` object also comes with several built-in plotting method to visualise the sensors readings:
- [`compute_psd()`](https://mne.tools/stable/generated/mne.io.Raw.html#mne.io.Raw.compute_psd) -  to compute the **power spectral density (PSD)** for each sensor type. *Power spectral density (PSD)* is a way to describe how the power of a signal is distributed across different frequencies.
- [`plot()`](https://mne.tools/stable/generated/mne.io.Raw.html#mne.io.Raw.plot) - to plot the raw sensor traces or the `compute_psd()` data. In interactive Python sessions, `plot()` is interactive and allows scrolling, scaling, bad channel marking, annotations, projector toggling, etc.

> Note: Here in the PSD plot, we’ll only plot frequencies below 50 Hz

In [ ]:
# PSD plot
sample_raw.compute_psd(fmax=50).plot(picks="data", exclude="bads", amplitude=False)

In [ ]:
# plot raw sensor readings
sample_raw.plot(duration=5, n_channels=30)

## 2. Preprocessing

Now, raw brain/neuron signals (EEG/MEG/iEEG/LFP) are noisy and includes suppress environmental noise (e.g., 50/60 Hz mains), sensor drift, and muscle/eye/heart artifacts, and so the main neural activity readings fades out. Filtering and preprocessing are what turn “raw, messy voltages” into data you can trust for analysis and modeling.
<br />
MNE-python provides [`mne.preprocessing`](https://mne.tools/stable/api/preprocessing.html#module-mne.preprocessing) and [`mne.filter`](https://mne.tools/stable/api/preprocessing.html#module-mne.filter) submodules to support a variety of preprocessing approaches and techniques (maxwell filtering, signal-space projection, independent components analysis, filtering, downsampling, etc)
<br />
<br />
One way to clean up the sensor readings is by using **independent components analysis (ICA)**. 
<br />
Think of your sensors like several microphones in a noisy room: they record a jumble of sounds at once.  *ICA* tries to actually separate the voices so each track is mostly one thing — eye blinks, heartbeat, or a brain rhythm. Then you can mute the blink/heartbeat tracks and keep the brain track, making the data much cleaner. We filter first (remove slow drift and mains hum) so *ICA* sees clearer patterns and doesn’t waste effort on obvious noise. 
> Note: ICA is bit different than PCA (Principal Component Analysis), which is like pointing the mics to where the sound is strongest—useful for shrinking the data, but the voices are still mixed.

Luckily, MNE-Python has `mne.preprocessing.ICA`. Let's use it to clean up our data by performing and for brevity we’ll skip the steps that helped us determined which components best capture the artifacts and that we want to remove them. 
<br />
So, once we’re confident about which component(s) we want to remove, we pass them as the `exclude` parameter and then apply the *ICA* to the raw signal. The [`apply()`](https://mne.tools/stable/generated/mne.preprocessing.ICA.html#mne.preprocessing.ICA.apply) method requires the raw data to be loaded into memory (by default it’s only read from disk as-needed), so we’ll use [`load_data()`](https://mne.tools/stable/generated/mne.preprocessing.ICA.html#mne.preprocessing.ICA.apply) first.


In [ ]:
# set up the ICA
ica = mne.preprocessing.ICA(n_components=20, random_state=97, max_iter=800)
# fit our sample dataset
ica.fit(sample_raw)

# pick the properties we want to exclude,
# details on how we picked these are omitted here
ica.exclude = [1, 2] 
# plot those properties
ica.plot_properties(sample_raw, picks=ica.exclude)

We’ll also make a copy of the `Raw` object before loading the data on memory and applying the *ICA* so we can compare the signal before and after artifact removal side-by-side:

In [ ]:
orig_sample_raw = sample_raw.copy()

# load the data
sample_raw.load_data()
# apply ICA
ica.apply(sample_raw)

In [ ]:
# show some frontal channels to clearly illustrate the artifact removal
chs = [
    "MEG 0111",
    "MEG 0121",
    "MEG 0131",
    "MEG 0211",
    "MEG 0221",
    "MEG 0231",
    "MEG 0311",
    "MEG 0321",
    "MEG 0331",
    "MEG 1511",
    "MEG 1521",
    "MEG 1531",
    "EEG 001",
    "EEG 002",
    "EEG 003",
    "EEG 004",
    "EEG 005",
    "EEG 006",
    "EEG 007",
    "EEG 008",
]

# channels index
chan_idxs = [sample_raw.ch_names.index(ch) for ch in chs]
# plot the readings
orig_sample_raw.plot(order=chan_idxs, start=12, duration=4)
sample_raw.plot(order=chan_idxs, start=12, duration=4)

## Detecting experimental events

The sample dataset includes several *“STIM” channels* that recorded electrical signals sent from the *stimulus delivery computer* (as brief DC shifts / squarewave pulses). These pulses (often called *“triggers”*) are used in this dataset to mark experimental events: *stimulus onset*, *stimulus type*, and *participant response (button press)*.
<br/>
The individual STIM channels are combined onto a single channel, in such a way that voltage levels on that channel can be unambiguously decoded as a particular event type.
<br />
On older *Neuromag systems* (such as that used to record the sample data) this summation channel was called `STI 014`, so we can pass that channel name to the [`mne.find_events()`](https://mne.tools/stable/generated/mne.find_events.html#mne.find_events) function to recover the timing and identity of the stimulus events.


In [ ]:
stims_events = mne.find_events(sample_raw, stim_channel="STI 014")
# show the first 5
print("=================")
print(stims_events[:5])

The resulting events array above is an ordinary 3-column *NumPy array*, with *sample number* in the first column and *integer event ID* in the last column; the middle column is usually ignored.
<br />
Rather than keeping track of integer event IDs, we can provide an event dictionary that maps the integer IDs to experimental conditions or events. In this dataset, the mapping looks like this:
|Event ID|Condition|
|--------|---------|
| 1 | auditory stimulus (tone) to the left ear |
| 2 | auditory stimulus (tone) to the right ear |
| 3 | visual stimulus (checkerboard) to the left visual field |
| 4 | visual stimulus (checkerboard) to the right visual field |
| 5 | smiley face (catch trial) |
| 32 | subject button press |

In [ ]:
stim_event_dict = {
    "auditory/left": 1,
    "auditory/right": 2,
    "visual/left": 3,
    "visual/right": 4,
    "smiley": 5,
    "buttonpress": 32,
}

Let's now plot these events using `mne.viz.plot_events()` function to visualise the distribution of events across the duration of the recording.
> Note: Here we will also make use of the Info attribute to get the sampling frequency of the recording (so our x-axis will be in seconds instead of in samples).

In [ ]:
fig = mne.viz.plot_events(
    stims_events, event_id=stim_event_dict, sfreq=sample_raw.info["sfreq"], first_samp=sample_raw.first_samp
)

For paradigms that are not event-related (e.g., analysis of resting-state data), you can extract regularly spaced (possibly overlapping) spans of data by creating events using `mne.make_fixed_length_events()` and then proceeding with **epoching**.

> Note: Resting-state means recording brain activity while the person (or animal) isn’t doing any specific task—usually just sitting quietly with eyes open (fixating) or eyes closed, relaxed and still. In this condition, the signals reflect the brain’s spontaneous rhythms and how regions naturally interact (e.g., the alpha rhythm around 8–12 Hz often grows with eyes closed). Researchers use resting-state data as a baseline, to study networks/connectivity, and to compare groups or conditions without the confounds of task performance.

### Epoching continuous data

An epoch is just a short slice of your long, continuous recording taken around a meaningful event—like “from 200 ms before a beep to 800 ms after it.” We epoch continuous data so we can line up many repeats of the same event, throw out slices with big artifacts (blinks, movement), do baseline correction using the pre-event period, and then average or compare conditions. This boosts the true, time-locked brain response while unrelated noise cancels out, makes statistics straightforward, and keeps data sizes manageable. For resting-state or very slow trends you might stay continuous, but for event-related questions, epoching is the standard way to get clean, comparable trials.
<br />
Now because each epoch is built from many channels at once, a single big artifact on any one channel can ruin the whole trial and bias your averages, stats, covariance, or decoding. Reject criteria are simple, objective limits (e.g., “drop the epoch if any channel’s peak-to-peak is too large, or if a channel goes flat/clipped”) that automatically exclude bad trials so only clean, comparable data remain. They’re set per channel type because sensors have different units and typical amplitudes (EEG vs MEG vs EOG), and using type-appropriate thresholds keeps you from over- or under-rejecting. Net effect: fewer outliers, better SNR, and more trustworthy results across all channels.
<br />
<br />
In MNE-Python, the `Raw` object and the *events array* are the bare minimum needed to create an [`Epochs`](https://mne.tools/stable/generated/mne.Epochs.html#mne.Epochs) object, which we create with the *`Epochs` class constructor*.
<br />
In our example, we’ll also specify some data quality constraints: we’ll reject any *epoch* where *peak-to-peak signal amplitude* is beyond reasonable limits for that *channel type*. This is done with a *rejection dictionary*; you may include or omit thresholds for any of the channel types present in your data.

> Note: The values given here are reasonable for this particular dataset, but may need to be adapted for different hardware or recording conditions. For a more automated approach, consider using the [autoreject package](http://autoreject.github.io/).

In [ ]:
reject_criteria = dict(
    mag=4000e-15, # 400 ft
    grad=1000e-13, # 4000 ft/cm
    eeg=150e-6, # 1500 µV
    eog=250e-6 # 150 µV
) 

Now, let's create an `Epochs` object by passing the event dictionary as the `event_id` parameter (so we can work with easy-to-pool event labels instead of the integer event IDs), and specify `tmin` and `tmax` params (the time relative to each event at which to start and end each epoch).

> Note: By default `Raw` and `Epochs` data aren’t loaded into memory (they’re accessed from disk only when needed), but here we’ll force loading into memory using the `preload=True` parameter so that we can see the results of the rejection criteria being applied

In [ ]:
epochs = mne.Epochs(
    sample_raw,
    stims_events,
    event_id=stim_event_dict,
    tmin=-0.2,
    tmax=0.5,
    reject=reject_criteria,
    preload=True
)

Now in general, you need to balance (equalize) epoch counts so your results aren’t driven by how many trials each subgroup contributes, but by the true effect you care about. If one subgroup (e.g., “left”) has more clean epochs than another (“right”), its average is less noisy and will dominate pooled averages, inflate SNR, skew statistics, and bias decoders simply because it has more data, not stronger neural responses. Equalizing counts (or otherwise weighting) keeps noise levels comparable across groups, avoids class-imbalance bias, makes contrasts fair, and ensures any difference you see reflects the underlying physiology—not unequal trial numbers or artifact-related dropout.
<br/>
<br />
Let's say here, we care about the *auditory* and *visual* groups and we want to compare auditory versus visual responses. So, we’ll pool across *left/right stimulus presentations* and to avoid biasing our signals to the left or right, we’ll use `epochs.equalize_event_counts()` first to randomly sample epochs from each condition to match the number of epochs present in the condition with the fewest good epochs.
<br />
In simpler words, because when you “pool” left and right trials into a single auditory (or visual) set, you can accidentally overweight whichever side has more clean epochs after rejection (e.g., 120 left vs 80 right). If you average those together as-is, the pooled response is pulled toward the dominant side, mixing true modality effects with left/right lateralization and unequal noise. equalize_event_counts fixes this by randomly downsampling each subgroup so they contribute equally (e.g., 80 left + 80 right), making your auditory–visual comparison fair, keeping signal-to-noise comparable across conditions, and avoiding class-imbalance issues for decoding.

In [ ]:
care_about = ["auditory/left", "auditory/right", "visual/left", "visual/right"]
epochs.equalize_event_counts(care_about) # it's an in-place operation

aud_epochs = epochs["auditory"]
vis_epochs = epochs["visual"]

# free up memory
del sample_raw, epochs

`Epochs` object also comes with a built-in plot function `plot_image()`, which shows each epoch as one row of an image map, with color representing signal magnitude; the average evoked response and the sensor location:

In [ ]:
aud_epochs.plot_image(picks=["MEG 1332", "EEG 021"])
vis_epochs.plot_image(picks=["MEG 1332", "EEG 021"])

### Time-frequency analysis

Time-frequency analysis is like using a music equalizer that shows which notes (frequencies) are playing at what moments in your brain data. Regular averages only show the overall waveform and can miss short-lived “bursts” or changes that aren’t perfectly time-locked to the event, so they get washed out. By breaking the signal into time and frequency, you can see, for example, a brief alpha dip right after a cue or a beta rebound later—when it happens and how strong it is. This makes it easier to detect transient rhythms, compare conditions fairly (with a baseline), and spot artifacts that sit in particular frequency bands.
<br />
<br />
The `mne.time_frequency` and `Epochs.compute_tfr()` submodule provides implementations of several algorithms to compute *time-frequency representations*, *power spectral density*, and *cross-spectral density*.
<br />
Here, for example, we’ll compute the induced power at different frequencies and times, using *Morlet wavelets*, for the *auditory epochs*.

> Note: On this dataset the result is not especially informative (it just shows the evoked “auditory N100” response); see [here](https://mne.tools/stable/auto_tutorials/time-freq/20_sensors_time_frequency.html#inter-trial-coherence) for a more extended example on a dataset with richer frequency content.

In [ ]:
frequencise = np.arange(
    start=7, stop=30, step=3
)
induced_power = aud_epochs.compute_tfr(
    "morlet", n_cycles=2, return_itc=False, freqs=frequencise, decim=3, average=True
)
induced_power.plot(["MEG 1332"])

### Estimating evoked responses
*Evoked responses (ERPs/ERFs)* are the brain’s consistent, time-locked reactions to a *stimulus* (like a beep or flash). We estimate them by averaging many aligned *epochs*, which cancels random noise and boosts the tiny, repeatable signal, so we can clearly see when the brain responds (latency) and how strongly (amplitude). This cleaner waveform lets us compare conditions or groups (e.g., visual vs auditory, patients vs controls), identify classic components (P1/N1/P300 or M100), do reliable statistics and baseline correction, and even get better source estimates—all because the averaging step greatly improves signal-to-noise for time-locked activity.
<br />
<br />
So for our example, now that we have our conditions in `aud_epochs` and `vis_epochs`, we can get an estimate of evoked responses to *auditory* versus *visual* stimuli by averaging together the epochs in each condition. This is as simple as calling the [`average()`](https://mne.tools/stable/generated/mne.Epochs.html#mne.Epochs.average) method on the `Epochs` object, and then using a function from the [`mne.viz`](https://mne.tools/stable/api/visualization.html#module-mne.viz) module to compare the global field power for each sensor type of the two [`Evoked`](https://mne.tools/stable/generated/mne.Evoked.html#mne.Evoked) objects, using the `.plot_compare_evokeds()` function:

In [ ]:
aud_evoked = aud_epochs.average()
vis_evoked = vis_epochs.average()

mne.viz.plot_compare_evokeds(
    dict(auditory=aud_evoked, visual=vis_evoked),
    legend="upper left",
    show_sensors="upper right"
)

We can also get a more detailed view of each `Evoked` object using other plotting methods such as `.plot_joint` or `.plot_topomap`. 
<br />
Here we’ll examine just the *EEG channels*, and see the classic auditory evoked *N100-P200* pattern over *dorso-frontal electrodes*, then plot scalp topographies at some additional arbitrary times:

In [ ]:
aud_evoked.plot_joint(picks="eeg")
aud_evoked.plot_topomap(
    times=[0.0, 0.08, 0.1, 0.12, 0.2],
    ch_type="eeg"
)

`Evoked` objects can also be combined to show contrasts between conditions, using the `mne.combine_evoked()` function. A simple difference can be generated by passing `weights=[1, -1]`. We’ll then plot the difference wave at each sensor using `plot_topo`.
<br />
In other words, `combine_evoked` with `weights=[1, -1]` makes a difference wave: *Condition A* minus *Condition B* at every time point and sensor. This contrast cancels what both conditions share and highlights what’s truly different — so you can see when the effect happens and where on the scalp/helmet it’s strongest (`plot_topo`). 

In [ ]:
evoked_diff = mne.combine_evoked([aud_evoked, vis_evoked], weights=[1, -1])
evoked_diff.pick(picks="mag").plot_topo(color="r", legend=False)

### Inverse modeling

Inverse modeling answers the “where in the brain did this signal come from?” question. Sensors (EEG/MEG) pick up mixtures of many sources; scalp maps alone can’t tell you the true generators because many different source patterns could produce the same sensor pattern (the problem is ill-posed). Inverse modeling uses a forward/physics head model (anatomy + conductivities) plus constraints/regularization (e.g., minimum-norm, dSPM, sLORETA, beamformers) to estimate activity on the cortex that best explains the sensor data. You get time courses per brain location/ROI, reduced sensor mixing/field-spread issues, better anatomical interpretability, and cleaner group stats and connectivity done in source space rather than at sensors.
<br />
<br />
So, for our example we can estimate the origins of the evoked activity by projecting the sensor data into this subject’s [source space](https://mne.tools/stable/documentation/glossary.html#term-source-space) (a set of points either on the cortical surface or within the cortical volume of that subject, as estimated by structural MRI scans).
<br />
*MNE-Python* supports lots of ways of doing this (*dynamic statistical parametric mapping, dipole fitting, beamformers*, etc.); here we’ll use **minimum-norm estimation (MNE)** to generate a continuous map of activation constrained to the cortical surface. *MNE* uses a *linear [inverse operator](https://mne.tools/stable/documentation/glossary.html#term-inverse-operator)* to project EEG+MEG sensor measurements into the source space. 
<br />
The *inverse operator* is computed from the [*forward solution*](https://mne.tools/stable/documentation/glossary.html#term-forward-solution) for this subject and an estimate of [the covariance of sensor measurements](https://mne.tools/stable/auto_tutorials/forward/90_compute_covariance.html#tut-compute-covariance).
<br />
<br />
For this tutorial we’ll skip those computational steps and load a pre-computed inverse operator from disk (it’s included with the *sample data*). Because this “inverse problem” is underdetermined/ill-posed (there is no unique solution), here we further constrain the solution by providing a regularization parameter specifying the relative smoothness of the current estimates in terms of a signal-to-noise ratio (where “noise” here is akin to baseline activity level across all of cortex).

In [ ]:
# path to the inverse operator
inverse_operator_file_path = sample_data_path.split("/")
inverse_operator_file_path[-1] = "sample_audvis-meg-oct-6-meg-inv.fif"
inverse_operator_file_path = "/".join(inverse_operator_file_path)

assert os.path.exists(inverse_operator_file_path)
print(inverse_operator_file_path)

# load the inverse operator
inv_operator = mne.minimum_norm.read_inverse_operator(inverse_operator_file_path)
# set signal-to-noise (SNR) to compute regularization paramter (λ²)
snr = 3.0
lambda2 = 1.0 / snr**2

# generate the source time course (STC)
stc = mne.minimum_norm.apply_inverse(
    vis_evoked, inv_operator, lambda2=lambda2, method="MNE"  # or dSPM, sLORETA, eLORETA
)

Finally, in order to plot the source estimate on the subject’s cortical surface we’ll also need the path to the sample subject’s structural MRI files (the `subjects_dir`):

In [ ]:
# path to subjects' MRI files
subjects_dir = sample_data_path.split("/")
subjects_dir = subjects_dir[:2]
subjects_dir[-1] = "subjects"
subjects_dir = "/".join(subjects_dir)

assert(os.path.exists(subjects_dir))
print(subjects_dir)

In [ ]:
# plot the STC
stc.plot(
    initial_time=0.1, hemi="split", views=["lat", "med"], subjects_dir=subjects_dir
)